In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import tensorflow as tf
import matplotlib.pyplot as plt
import requests 
import matplotlib.image as mpimg
from sklearn import preprocessing

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../"))

# Any results you write to the current directory are saved as output.

['classes-trainable.csv', 'train_human_labels.csv', 'stage_1_sample_submission.csv', 'stage_1_test_images', 'tuning_labels.csv', 'stage_1_attributions.csv', 'train_bounding_boxes.csv', 'class-descriptions.csv', 'train_machine_labels.csv']
['config', 'working', 'lib', 'input']


In [2]:
#getting trainable classes from left file and getting label description for those classes only
cl=pd.merge(pd.read_csv('../input/classes-trainable.csv'),
            pd.read_csv('../input/classes-trainable.csv'),
           how='left', on=['label_code'])

#Stacks up both table. train_labels will contain 4 rows (ImageID,Source,LabelName,Confidence) for both sources human and machine.
train_labels = pd.concat((pd.read_csv('../input/train_machine_labels.csv'), 
                          pd.read_csv('../input/train_human_labels.csv')), 
                         axis=0, ignore_index=True)

train_bb = pd.read_csv('../input/train_bounding_boxes.csv')
train_tl = pd.read_csv('../input/tuning_labels.csv')

In [3]:
train_bb.set_index('ImageID',inplace=True)
train_bb.head()

,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
ImageID,,,,,,,,,,,,
000002b66c9c498e,xclick,/m/01g317,1,0.012500,0.195312,0.148438,0.587500,0,1,0,0,0
000002b66c9c498e,xclick,/m/01g317,1,0.025000,0.276563,0.714063,0.948438,0,1,0,0,0
000002b66c9c498e,xclick,/m/01g317,1,0.151562,0.310937,0.198437,0.590625,1,0,0,0,0
000002b66c9c498e,xclick,/m/01g317,1,0.256250,0.429688,0.651563,0.925000,1,0,0,0,0
000002b66c9c498e,xclick,/m/01g317,1,0.257812,0.346875,0.235938,0.385938,1,0,0,0,0


In [ ]:
test = pd.read_csv('../input/stage_1_sample_submission.csv')
test = pd.merge(test, pd.read_csv('../input/stage_1_attributions.csv'), how='left', on=['image_id'])
test.head()

In [ ]:
test['path'] = test['image_id'].map(lambda x: '../input/stage_1_test_images/' + str(x) + '.jpg')
test.head() #test now contains also path for their respective image_id

In [4]:
import bq_helper
from bq_helper import BigQueryHelper

open_images = bq_helper.BigQueryHelper(active_project="bigquery-public-data", 
                                       dataset_name="open_images")
open_images.list_tables()

['annotations_bbox', 'dict', 'images', 'labels']

In [5]:
# getting URL from open_images dataset for image_id in train_labels 
image_ids = train_labels.ImageID.unique()

for i in range(0, len(image_ids), 20):
    open_images = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="open_images")
    query = "SELECT image_id, thumbnail_300k_url, original_url, original_landing_url  FROM `bigquery-public-data.open_images.images` WHERE image_id IN ('" + "', '".join(image_ids[i : i+1000]) + "');"
    df = open_images.query_to_pandas_safe(query, max_gb_scanned=10)
    df.to_csv('image_urls' + str(i).zfill(10) + '.csv')
    break

In [6]:
print(df.shape) 
urls=df.values
df.head()

(1000, 4)


,image_id,thumbnail_300k_url,original_url,original_landing_url
0,0003bf6ad3a94983,None,https://farm4.staticflickr.com/4033/4222511969...,https://www.flickr.com/photos/lum1neuz/4222511969
1,0005bda112291d3b,None,https://c4.staticflickr.com/1/252/521666000_43...,https://www.flickr.com/photos/derricksphotos/5...
2,00030b8c7652041d,None,https://c6.staticflickr.com/1/13/18460961_6b60...,https://www.flickr.com/photos/veritatem/18460961
3,00081db9138ff646,None,https://farm7.staticflickr.com/1064/565353386_...,https://www.flickr.com/photos/snapr/565353386
4,00050546c36b308c,None,https://c5.staticflickr.com/5/4022/4499952655_...,https://www.flickr.com/photos/anne-cathrine_ny...


In [7]:
num_pictures=128
image_size=64
num_channels=3


In [8]:

def get_data_from_url(image_id,url,image_index,image_data,labels):
    r = requests.get(url)
    with open("img.jpg",'wb') as f: 
        f.write(r.content) 
        
        file_data=mpimg.imread("../working/img.jpg")
        
        
        bb=train_bb.loc[image_id,['YMin','XMin','YMax','XMax']].values
        if len(bb.shape)==1:
            box=np.ndarray(shape=(1,4))
            box[0,:]=bb[0]
            bb=box
        lb=train_bb.loc[image_id,['LabelName']]
        label_gate=0
        if(lb.shape[0]==1):
            lb=lb.values
            gate=1
        else:
            lb=lb['LabelName'].tolist()
        
        y=np.ndarray(shape=(1,file_data.shape[0],file_data.shape[1],3))
        y[0,:,:,:]=file_data[:,:,0:3]
        
        cropped=tf.image.crop_and_resize(image=y,boxes=bb,box_ind=[0]*bb.shape[0],crop_size=[image_size,image_size],method='bilinear')
        sess=tf.Session()
        with sess.as_default():
            tensor=cropped
            cropped=tensor.eval()
        n=cropped.shape[0]
        for i in range(n):
            image_data[image_index,:,:,:]=cropped[i,:,:,:]
            image_index=image_index+1
            if label_gate==0:
                labels.append(lb[i])
            else:
                labels.extend(lb)
            if image_index%50==0:
                print(image_index)
            if image_index>=num_pictures:
                break
        
        f.close()
        os.remove("../working/img.jpg")
        return image_index,image_data,labels

In [9]:
def get_next_batch(url_index):
    image_index=0
    image_data=np.ndarray(shape=(num_pictures,image_size,image_size,num_channels))
    labels=[]
    k=url_index
    for i in range(k,k+num_pictures):
        image_id=urls[i,0]
        url=urls[i,2]
        image_index,image_data,labels=get_data_from_url(image_id,url,image_index,image_data,labels)
        url_index=url_index+1
        if image_index>=num_pictures:
            break
    
    
    labels=np.asarray(labels)
    

    random=np.arange(num_pictures)
    np.random.shuffle(random)
    image_data=image_data[random]
    labels=labels[random]
    
    
    return image_data,labels,url_index

In [10]:
all_labels=train_bb['LabelName'].values.tolist()
le = preprocessing.LabelEncoder()
training_le = le.fit(all_labels)
num_classes=le.classes_.shape[0]

In [11]:
def get_encoded_labels(labels):
    training_labels_encoded = training_le.transform(labels)
    return training_labels_encoded

In [12]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Add, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Concatenate
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [ ]:
def identity_block(X, f, filters):
    
    F1, F2, F3 = filters
     
    X_shortcut = X
   
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = X = Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)

    X = layers.Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [ ]:

def convolutional_block(X, f, filters,s = 2):
    
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv2D(F1, (1, 1), strides = (s,s), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2,(f,f),strides=(1,1),padding='same',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(F3,(1,1),strides=(1,1),padding='valid',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)

    X_shortcut = Conv2D(F3,(1,1),strides=(s,s),padding='valid',kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    X = layers.Add()([X,X_shortcut])
    X = Activation('relu')(X)
    return X

In [ ]:

def ResNet50(input_shape, classes):
    
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)
    
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X,f=3,filters=[128,128,512],s=2)
    X = identity_block(X,3,[128,128,512])
    X = identity_block(X,3,[128,128,512])
    X = identity_block(X,3,[128,128,512])

    X = convolutional_block(X,f=3,filters=[256,256,1024],s=2)
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])

    X = convolutional_block(X,f=3,filters=[512,512,2048],s=2)
    X = identity_block(X,3,[512,512,2048])
    X = identity_block(X,3,[512,512,2048])

    X = AveragePooling2D((2,2))(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
model = ResNet50(input_shape = (image_size, image_size, 3), classes=num_classes)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
url_index=0
times=150
for i in range(0,times):
    print(url_index)
    image_data,labels,url_index=get_next_batch(url_index)
    Y=get_encoded_labels(labels)
    model.fit(image_data, Y, epochs = 3, batch_size = 32)